In [112]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import json
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import time
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
from sklearn.preprocessing import MinMaxScaler

In [113]:
pd.options.mode.copy_on_write = True 

In [114]:
steam_games_df_base = pd.read_json('steam_games_df.json', lines=True)

reviews_df_base = pd.read_json('reviews_df.json', lines=True)

user_items_df_base = pd.read_json('user_items_df.json', lines=True)

In [115]:
steam_games_df = steam_games_df_base

reviews_df = reviews_df_base

user_items_df = user_items_df_base

# Preprocesamiento

In [116]:
#steam games
steam_games_df = steam_games_df[steam_games_df['item_id'].notna()]
steam_games_df.reset_index(drop=True, inplace=True)
steam_games_df['item_id'] = steam_games_df['item_id'].astype(int)

In [117]:
steam_games_df

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,item_id,developer,sentiment
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,None
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,None
4,,[],Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32128,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",1.49,http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS""",None
32129,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",4.24,http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada,None
32130,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",1.39,http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich,None
32131,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",NaN,http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns",1 user reviews


In [118]:
#reviews 
reviews_df = reviews_df[reviews_df['item_id'].notna()]
reviews_df.reset_index(drop=True, inplace=True)
reviews_df['item_id'] = reviews_df['item_id'].astype(int)
#Si no esta en nuestra base de steam games. 
reviews_df = reviews_df[reviews_df['item_id'].isin(steam_games_df['item_id'])]
reviews_df = reviews_df.drop(columns=['funny', 'posted', 'last_edited', 'helpful', 'user_url'], axis=1)

In [119]:
reviews_df

,item_id,recommend,review,user_id
0,1250,1.0,Simple yet with great replayability. In my opi...,76561197970982479
1,22200,1.0,It's unique and worth a playthrough.,76561197970982479
4,227300,1.0,For a simple (it's actually not all that simpl...,js41637
5,239030,1.0,Very fun little game to play when your bored o...,js41637
6,248820,1.0,A suitably punishing roguelike platformer. Wi...,evcentric
...,...,...,...,...
59300,70,1.0,a must have classic from steam definitely wort...,76561198312638244
59301,362890,1.0,this game is a perfect remake of the original ...,76561198312638244
59302,273110,1.0,had so much fun plaing this and collecting res...,LydiaMorley
59303,730,1.0,:D,LydiaMorley


In [120]:
# Este cambio se hizo antes -> steam_games_df.rename(columns={'id': 'item_id'}, inplace=True)
#User Items
user_items_df = user_items_df[user_items_df['item_id'].isin(steam_games_df['item_id'])]
user_items_df['item_id'] = user_items_df['item_id'].astype(int)
user_items_df = user_items_df.drop(columns=['user_url'], axis=1)

In [121]:
user_items_df

,item_id,item_name,playtime_forever,playtime_2weeks,user_id
0,10,Counter-Strike,6.0,0.0,76561197970982479
1,20,Team Fortress Classic,0.0,0.0,76561197970982479
2,30,Day of Defeat,7.0,0.0,76561197970982479
3,40,Deathmatch Classic,0.0,0.0,76561197970982479
4,50,Half-Life: Opposing Force,0.0,0.0,76561197970982479
...,...,...,...,...,...
5170008,227940,Heroes & Generals,43.0,43.0,76561198329548331
5170009,346330,BrainBread 2,0.0,0.0,76561198329548331
5170010,373330,All Is Dust,0.0,0.0,76561198329548331
5170011,388490,One Way To Die: Steam Edition,3.0,3.0,76561198329548331


In [122]:
def sigmoid(x, scale=1):
    return 1 / (1 + np.exp(-x * scale))
user_items_df['log_ptime'] = np.log1p(user_items_df['playtime_forever'])
user_items_df['exp_ptime_2week'] = sigmoid(user_items_df['playtime_2weeks'], scale=0.3) 
max_value = 10 
user_items_df['exp_ptime_2week'] *= max_value

In [123]:
scaler_playtime = MinMaxScaler(feature_range=(0, 5))
user_items_df['log_ptime_scaled'] = scaler_playtime.fit_transform(user_items_df[['log_ptime']].values.reshape(-1, 1))

scaler_lastweek = MinMaxScaler(feature_range=(0, 5))
user_items_df['exp_ptime_2week_scaled'] = scaler_lastweek.fit_transform(user_items_df[['exp_ptime_2week']].values.reshape(-1, 1))

In [124]:
n=0.6 # Peso para el tiempo de juego en las últimas dos semanas
m=0.4 # Peso para el tiempo de juego total
user_items_df['combined_rating'] = n * user_items_df['exp_ptime_2week_scaled'] + m * user_items_df['log_ptime_scaled']

In [125]:
user_items_df = pd.merge(user_items_df, reviews_df[['user_id', 'item_id', 'recommend']], on=['user_id', 'item_id'], how='left')
#result = pd.merge(user_items_df, reviews_df, on=['user_id', 'item_id'],how='left')


In [182]:
user_items_df

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,log_ptime,exp_ptime_2week,log_ptime_scaled,exp_ptime_2week_scaled,combined_rating,recommend,weighted_rating
0,10,Counter-Strike,6.0,0.0,76561197970982479,1.945910,5.000000,0.727522,0.000000,0.291009,NaN,0.291009
1,20,Team Fortress Classic,0.0,0.0,76561197970982479,0.000000,5.000000,0.000000,0.000000,0.000000,NaN,0.000000
2,30,Day of Defeat,7.0,0.0,76561197970982479,2.079442,5.000000,0.777446,0.000000,0.310978,NaN,0.310978
3,40,Deathmatch Classic,0.0,0.0,76561197970982479,0.000000,5.000000,0.000000,0.000000,0.000000,NaN,0.000000
4,50,Half-Life: Opposing Force,0.0,0.0,76561197970982479,0.000000,5.000000,0.000000,0.000000,0.000000,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
4295534,227940,Heroes & Generals,43.0,43.0,76561198329548331,3.784190,9.999975,1.414804,4.999975,3.565907,NaN,3.565907
4295535,346330,BrainBread 2,0.0,0.0,76561198329548331,0.000000,5.000000,0.000000,0.000000,0.000000,NaN,0.000000
4295536,373330,All Is Dust,0.0,0.0,76561198329548331,0.000000,5.000000,0.000000,0.000000,0.000000,NaN,0.000000
4295537,388490,One Way To Die: Steam Edition,3.0,3.0,76561198329548331,1.386294,7.109495,0.518297,2.109495,1.473016,NaN,1.473016


In [126]:
#result

In [127]:
user_items_df['weighted_rating'] = user_items_df['combined_rating'] * (1 + user_items_df['recommend'].fillna(0))

In [128]:
user_item_matrix = user_items_df.pivot_table(values='weighted_rating', index='item_id', columns='user_id', fill_value=0)

In [129]:
user_item_matrix

user_id,--000--,--ace--,--ionex--,-2SV-vuLB-Kg,-404PageNotFound-,-AnimeIsMyThing-,-Azsael-,-Beave-,-Encore-,-GM-Dragon,...,zyxwvutsrqponm,zyzzchris,zzbrunozz,zzeee,zzjang2,zzonci,zzoptimuszz,zzydrax,zzyfo,zzzmidmiss
item_id,,,,,,,,,,,,,,,,,,,,,
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.404986,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.267956,0.0
20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.617209,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527900,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
528580,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
528660,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [155]:
user_item_matrix.shape

(9192, 68712)

In [142]:
from scipy.sparse import csc_matrix

sparse_user_item_matrix = csc_matrix(user_item_matrix.values)


In [157]:
U, sigma, VT = svds(sparse_user_item_matrix, k=100)  
sigma = np.diag(sigma)

# Compute the dot product to get the approximated original matrix
predicted_ratings = np.dot(np.dot(U, sigma), VT)

# Converting the reconstructed matrix back to a DataFrame
predicted_ratings_df = pd.DataFrame(predicted_ratings, index=user_item_matrix.index, columns=user_item_matrix.columns)

In [158]:

# Flatten the matrices to get vectors of all ratings
actual_ratings = user_item_matrix.values.flatten()
predicted_ratings = predicted_ratings_df.values.flatten()

# Only consider non-zero elements
mask = actual_ratings > 0
actual_ratings = actual_ratings[mask]
predicted_ratings = predicted_ratings[mask]

# Calculate RMSE
rmse_svd = sqrt(mean_squared_error(actual_ratings, predicted_ratings))
print("RMSE:", rmse_svd)

RMSE: 0.6734004342478984


In [169]:
def recommend_items_svd(user_id, original_matrix, predictions_df, num_recommendations=5):
    # Find the column for the user in the predictions matrix
    user_col_number = original_matrix.columns.get_loc(user_id)
    
    # Get actual user data and apply mask to filter out items already rated (non-zero ratings)
    user_actual_ratings = original_matrix[user_id]
    non_zero_mask = user_actual_ratings > 0  # True for items the user has rated
    
    # Generate predictions for the user, filtering out already rated items
    user_predictions = predictions_df.iloc[:, user_col_number].sort_values(ascending=False)
    recommended_items = user_predictions[~non_zero_mask].head(num_recommendations)
    
    return recommended_items

In [171]:
user_id = 'evcentric'  # Example user ID
recommendations = recommend_items_svd(user_id, user_item_matrix, predicted_ratings_df, num_recommendations=5)

recommendations_df = pd.DataFrame(recommendations).reset_index()
recommendations_df.columns = ['item_id', 'predicted_rating']
merged_df = recommendations_df.merge(steam_games_df, left_on='item_id', right_on='item_id', how='left')
merged_df

,item_id,predicted_rating,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,developer,sentiment
0,294100,0.656694,Ludeon Studios,"[Indie, Simulation, Strategy, Early Access]",RimWorld,RimWorld,http://store.steampowered.com/app/294100/RimWo...,2016-07-15,"[Early Access, Base Building, Survival, Strate...",NaN,http://steamcommunity.com/app/294100/reviews/?...,"[Single-player, Steam Workshop, Steam Cloud]",29.99,True,Ludeon Studios,Overwhelmingly Positive
1,359320,0.552884,,[],Elite Dangerous,None,http://store.steampowered.com/app/359320/Elite...,None,"[Space Sim, Space, Open World, Exploration, Sc...",NaN,http://steamcommunity.com/app/359320/reviews/?...,"[Single-player, Multi-player, MMO, Co-op, Part...",29.99,False,None,Mixed
2,225540,0.503546,Square Enix,"[Action, Adventure]",Just Cause™ 3,Just Cause™ 3,http://store.steampowered.com/app/225540/Just_...,2015-11-30,"[Open World, Action, Destruction, Third-Person...",NaN,http://steamcommunity.com/app/225540/reviews/?...,"[Single-player, Steam Achievements, Full contr...",49.99,False,Avalanche Studios,Mostly Positive
3,281990,0.477794,Paradox Interactive,"[Simulation, Strategy]",Stellaris,Stellaris,http://store.steampowered.com/app/281990/Stell...,2016-05-09,"[Space, Strategy, Grand Strategy, Sci-fi, 4X, ...",NaN,http://steamcommunity.com/app/281990/reviews/?...,"[Single-player, Multi-player, Cross-Platform M...",39.99,False,Paradox Development Studio,Very Positive
4,214950,0.456935,SEGA,[Strategy],Total War™: ROME II - Emperor Edition,Total War™: ROME II - Emperor Edition,http://store.steampowered.com/app/214950/Total...,2013-09-02,"[Strategy, Historical, Turn-Based Strategy, Gr...",NaN,http://steamcommunity.com/app/214950/reviews/?...,"[Single-player, Multi-player, Steam Trading Ca...",59.95,False,Creative Assembly,Mostly Positive


In [172]:
# def gen_recom_ground_truth_svd(num_recommendations=5):
#     recommendations = {}
#     ground_truth = {}
# 
#     for user_id in user_item_matrix.columns:
#         # Explicitly apply a mask to get non-zero rated items
#         user_ratings = user_item_matrix[user_id]
#         mask_non_zero = user_ratings > 0
#         ground_truth[user_id] = user_ratings[mask_non_zero].index.tolist()
# 
#         # Generate recommendations by first masking out known items
#         user_predictions = predicted_ratings_df.loc[:, user_id].sort_values(ascending=False)
#         # Filter out known items to focus only on items the user hasn't interacted with
#         recommendations[user_id] = user_predictions[~mask_non_zero].head(num_recommendations).index.tolist()
# 
#     return recommendations, ground_truth